# Bank Marketing - Analyse Exploratoire

## Contexte métier

**Problème** : Une banque portugaise mène des campagnes de télémarketing pour vendre des **dépôts à terme** (placements sécurisés avec taux fixe). L'objectif est de prédire quels clients sont susceptibles de souscrire.

**Contexte économique** : Les données couvrent **mai 2008 à novembre 2010**, période marquée par :
- La crise financière mondiale de 2008
- La crise de la dette souveraine européenne
- Une forte volatilité des marchés

Dans ce contexte, les dépôts à terme sont attractifs car ils offrent une **sécurité** face à l'incertitude des marchés.

**Enjeu business** : Optimiser les campagnes de télémarketing en ciblant les clients les plus susceptibles de souscrire, réduisant ainsi les coûts et augmentant le taux de conversion.

---

**Sources** :
- [UCI Bank Marketing Dataset](https://archive.ics.uci.edu/dataset/222/bank+marketing)
- Moro et al., 2014 - *A Data-Driven Approach to Predict the Success of Bank Telemarketing*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
%matplotlib inline

## 1. Chargement des données

On utilise le dataset **bank-additional-full.csv** qui contient les indicateurs économiques (vs bank-full.csv qui n'en a pas).

In [ ]:
df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep=';')
print(f"Dimensions : {df.shape[0]:,} lignes, {df.shape[1]} colonnes")
df.head()

## 2. Comprendre les features par catégorie métier

Les 20 features se regroupent en **4 catégories** :

| Catégorie | Features | Description |
|-----------|----------|-------------|
| **Client** | age, job, marital, education, default, housing, loan | Profil socio-démographique |
| **Contact** | contact, month, day_of_week, duration | Détails du dernier appel |
| **Historique** | campaign, pdays, previous, poutcome | Interactions passées |
| **Économie** | emp.var.rate, cons.price.idx, cons.conf.idx, euribor3m, nr.employed | Contexte macro-économique |

In [ ]:
# Organisation des features par catégorie métier
features_client = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan']
features_contact = ['contact', 'month', 'day_of_week', 'duration']
features_historique = ['campaign', 'pdays', 'previous', 'poutcome']
features_economie = ['emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']

print("Aperçu des types de données :")
df.dtypes

## 3. Variable cible : Déséquilibre des classes

Le télémarketing a un faible taux de conversion - c'est attendu dans ce domaine.

In [ ]:
target_counts = df['y'].value_counts()
target_pct = df['y'].value_counts(normalize=True) * 100

print("Distribution de la variable cible :")
print(f"  - Non souscrit : {target_counts['no']:,} ({target_pct['no']:.1f}%)")
print(f"  - Souscrit     : {target_counts['yes']:,} ({target_pct['yes']:.1f}%)")
print(f"\nRatio de déséquilibre : 1:{target_counts['no']//target_counts['yes']}")

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
colors = ['#c0392b', '#27ae60']
bars = ax.bar(['Non', 'Oui'], [target_counts['no'], target_counts['yes']], color=colors)
ax.set_title('Souscription au dépôt à terme', fontsize=14)
ax.set_ylabel('Nombre de clients')

# Ajouter les pourcentages sur les barres
for bar, pct in zip(bars, [target_pct['no'], target_pct['yes']]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 500, 
            f'{pct:.1f}%', ha='center', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️  IMPLICATION : Avec 88.7% de 'non', un modèle naïf qui prédit toujours 'non'")
print("    aurait 88.7% d'accuracy. L'accuracy seule n'est pas une bonne métrique ici.")

## 4. ⚠️ ALERTE : La variable `duration` (Data Leakage)

**Problème critique** : La durée de l'appel (`duration`) est connue **après** l'appel, pas avant. Or, après l'appel, on connaît déjà si le client a souscrit ou non.

Selon la documentation UCI :
> *"This attribute highly affects the output target. Yet, the duration is not known before a call is performed. This input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model."*

**Décision** : On analysera `duration` pour comprendre son impact, mais on l'exclura du modèle prédictif final.

In [ ]:
# Démonstration du problème de duration
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Distribution par classe
df[df['y'] == 'no']['duration'].hist(bins=50, alpha=0.6, label='Non souscrit', ax=axes[0], color='#c0392b')
df[df['y'] == 'yes']['duration'].hist(bins=50, alpha=0.6, label='Souscrit', ax=axes[0], color='#27ae60')
axes[0].set_title('Distribution de la durée par classe')
axes[0].set_xlabel('Durée (secondes)')
axes[0].legend()
axes[0].set_xlim(0, 2000)

# Durée moyenne par classe
duration_by_y = df.groupby('y')['duration'].mean()
axes[1].bar(['Non', 'Oui'], [duration_by_y['no'], duration_by_y['yes']], color=colors)
axes[1].set_title('Durée moyenne par classe')
axes[1].set_ylabel('Durée moyenne (secondes)')

plt.tight_layout()
plt.show()

print(f"Durée moyenne - Non souscrit : {duration_by_y['no']:.0f}s ({duration_by_y['no']/60:.1f} min)")
print(f"Durée moyenne - Souscrit     : {duration_by_y['yes']:.0f}s ({duration_by_y['yes']/60:.1f} min)")
print(f"\n→ Les appels qui aboutissent à une souscription durent ~2.5x plus longtemps.")
print("→ C'est logique : convaincre quelqu'un prend du temps !")

## 5. Profil client : Qui souscrit ?

In [ ]:
def taux_souscription(df, col):
    """Calcule le taux de souscription par catégorie"""
    return df.groupby(col)['y'].apply(lambda x: (x == 'yes').mean() * 100).sort_values(ascending=False)

# Taux de souscription par job
job_rates = taux_souscription(df, 'job')

fig, ax = plt.subplots(figsize=(10, 5))
bars = job_rates.plot(kind='bar', ax=ax, color='steelblue')
ax.axhline(y=target_pct['yes'], color='red', linestyle='--', linewidth=2, label=f'Moyenne globale ({target_pct["yes"]:.1f}%)')
ax.set_title('Taux de souscription par profession', fontsize=14)
ax.set_xlabel('')
ax.set_ylabel('Taux de souscription (%)')
ax.legend()
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

print("\n🎯 INSIGHTS MÉTIER :")
print(f"  - Les ÉTUDIANTS ont le meilleur taux ({job_rates['student']:.1f}%) → produit d'épargne attractif ?")
print(f"  - Les RETRAITÉS sont seconds ({job_rates['retired']:.1f}%) → recherche de placements sûrs")
print(f"  - Les OUVRIERS ont le plus faible taux ({job_rates['blue-collar']:.1f}%) → moins d'épargne disponible ?")

In [ ]:
# Analyse par âge
df['age_group'] = pd.cut(df['age'], bins=[0, 25, 35, 45, 55, 65, 100], 
                         labels=['18-25', '26-35', '36-45', '46-55', '56-65', '65+'])

age_rates = taux_souscription(df, 'age_group')

fig, ax = plt.subplots(figsize=(8, 4))
age_rates.plot(kind='bar', ax=ax, color='teal')
ax.axhline(y=target_pct['yes'], color='red', linestyle='--', label=f'Moyenne ({target_pct["yes"]:.1f}%)')
ax.set_title("Taux de souscription par tranche d'âge", fontsize=14)
ax.set_xlabel('')
ax.set_ylabel('Taux de souscription (%)')
ax.legend()
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n🎯 INSIGHT : Distribution en 'U' - les jeunes (<25) et les seniors (65+) souscrivent plus")
print("   → Les 30-50 ans ont d'autres priorités (immobilier, famille, etc.)")

In [ ]:
# Situation financière : prêts en cours
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

for ax, col, title in zip(axes, ['housing', 'loan', 'default'], 
                          ['Prêt immobilier', 'Prêt personnel', 'Défaut de crédit']):
    rates = taux_souscription(df, col)
    rates.plot(kind='bar', ax=ax, color=['#3498db', '#e74c3c', '#95a5a6'][:len(rates)])
    ax.set_title(title)
    ax.set_ylabel('Taux souscription (%)')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    ax.axhline(y=target_pct['yes'], color='red', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

print("\n🎯 INSIGHTS :")
print("  - Les clients SANS prêt immobilier souscrivent plus → plus de liquidités disponibles")
print("  - Les clients SANS prêt personnel souscrivent légèrement plus")
print("  - Le défaut de crédit a peu d'impact (mais très peu de cas 'yes')")

## 6. Historique des contacts : L'importance du timing

In [ ]:
# Taux de souscription par mois
month_order = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
df['month'] = pd.Categorical(df['month'], categories=month_order, ordered=True)

month_rates = df.groupby('month')['y'].apply(lambda x: (x == 'yes').mean() * 100)
month_counts = df.groupby('month').size()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Taux de souscription
month_rates.plot(kind='bar', ax=axes[0], color='darkgreen')
axes[0].axhline(y=target_pct['yes'], color='red', linestyle='--', label=f'Moyenne ({target_pct["yes"]:.1f}%)')
axes[0].set_title('Taux de souscription par mois', fontsize=14)
axes[0].set_ylabel('Taux (%)')
axes[0].legend()
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45)

# Volume d'appels
month_counts.plot(kind='bar', ax=axes[1], color='navy')
axes[1].set_title("Volume d'appels par mois", fontsize=14)
axes[1].set_ylabel("Nombre d'appels")
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45)

plt.tight_layout()
plt.show()

print("\n🎯 INSIGHTS STRATÉGIQUES :")
print("  - MARS, SEPTEMBRE, OCTOBRE, DÉCEMBRE ont les meilleurs taux (>40%)")
print("  - MAI a le plus d'appels mais un taux moyen → campagne de masse peu efficace")
print("  - Recommandation : concentrer les efforts en automne et fin d'année")

In [ ]:
# Impact du résultat de la campagne précédente
poutcome_rates = taux_souscription(df, 'poutcome')

fig, ax = plt.subplots(figsize=(8, 4))
colors_poutcome = {'success': '#27ae60', 'failure': '#c0392b', 'nonexistent': '#7f8c8d'}
poutcome_rates.plot(kind='bar', ax=ax, color=[colors_poutcome.get(x, 'gray') for x in poutcome_rates.index])
ax.set_title('Taux de souscription selon le résultat de la campagne précédente', fontsize=14)
ax.set_ylabel('Taux (%)')
ax.axhline(y=target_pct['yes'], color='red', linestyle='--')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print(f"\n🎯 INSIGHT CLÉ : Si la campagne précédente était un SUCCÈS → {poutcome_rates['success']:.0f}% de souscription !")
print("   → La fidélisation est plus efficace que la prospection")
print(f"   → 'nonexistent' (jamais contacté) : seulement {poutcome_rates['nonexistent']:.1f}%")

In [ ]:
# Nombre de contacts pendant la campagne
df['campaign_group'] = pd.cut(df['campaign'], bins=[0, 1, 2, 3, 5, 10, 100], 
                              labels=['1', '2', '3', '4-5', '6-10', '>10'])

campaign_rates = taux_souscription(df, 'campaign_group')

fig, ax = plt.subplots(figsize=(8, 4))
campaign_rates.plot(kind='bar', ax=ax, color='purple')
ax.set_title('Taux de souscription selon le nombre de contacts', fontsize=14)
ax.set_xlabel('Nombre de contacts pendant la campagne')
ax.set_ylabel('Taux (%)')
ax.axhline(y=target_pct['yes'], color='red', linestyle='--')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n🎯 INSIGHT : Le taux diminue avec le nombre de contacts")
print("   → Insister ne sert à rien après 3-4 appels")
print("   → Coût/bénéfice : mieux vaut passer au client suivant")

## 7. Contexte économique : L'impact de la crise

Les 5 indicateurs économiques capturent le contexte macro de 2008-2010 :
- **emp.var.rate** : Variation du taux d'emploi (trimestriel)
- **cons.price.idx** : Indice des prix à la consommation (inflation)
- **cons.conf.idx** : Indice de confiance des consommateurs
- **euribor3m** : Taux interbancaire à 3 mois (coût du crédit)
- **nr.employed** : Nombre d'employés (indicateur de santé économique)

In [ ]:
# Corrélation des indicateurs économiques avec la souscription
df['y_numeric'] = (df['y'] == 'yes').astype(int)

correlations = df[features_economie + ['y_numeric']].corr()['y_numeric'].drop('y_numeric').sort_values()

fig, ax = plt.subplots(figsize=(8, 4))
colors_corr = ['#27ae60' if x > 0 else '#c0392b' for x in correlations]
correlations.plot(kind='barh', ax=ax, color=colors_corr)
ax.set_title('Corrélation des indicateurs économiques avec la souscription', fontsize=14)
ax.set_xlabel('Corrélation')
ax.axvline(x=0, color='black', linewidth=0.5)
plt.tight_layout()
plt.show()

print("\n🎯 INTERPRÉTATION ÉCONOMIQUE :")
print("  - euribor3m NÉGATIF : quand les taux sont bas, les dépôts à terme sont moins attractifs")
print("  - nr.employed NÉGATIF : en période de plein emploi, moins besoin d'épargne de précaution")
print("  - emp.var.rate NÉGATIF : quand l'emploi augmente, les gens investissent ailleurs")
print("\n→ Paradoxe : les clients souscrivent PLUS en période d'incertitude économique")
print("→ Le dépôt à terme est un produit de 'refuge' en temps de crise")

In [ ]:
# Évolution de l'euribor3m et du taux de souscription
euribor_groups = pd.qcut(df['euribor3m'], q=5, labels=['Très bas', 'Bas', 'Moyen', 'Élevé', 'Très élevé'])
euribor_rates = df.groupby(euribor_groups)['y'].apply(lambda x: (x == 'yes').mean() * 100)

fig, ax = plt.subplots(figsize=(8, 4))
euribor_rates.plot(kind='bar', ax=ax, color='darkorange')
ax.set_title('Taux de souscription selon le niveau Euribor 3 mois', fontsize=14)
ax.set_ylabel('Taux (%)')
ax.set_xlabel('Niveau Euribor')
ax.axhline(y=target_pct['yes'], color='red', linestyle='--')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

print("\n🎯 INSIGHT : Quand l'Euribor est BAS, le taux de souscription est ÉLEVÉ (>20%)")
print("   → Semble contre-intuitif : taux bas = rendement du dépôt plus faible")
print("   → Mais : taux bas = crise = recherche de sécurité !")

## 8. Valeurs manquantes et 'unknown'

In [ ]:
# Pas de NaN, mais des 'unknown'
print("Valeurs NaN :", df.isnull().sum().sum())
print("\nValeurs 'unknown' par colonne :")

for col in df.select_dtypes(include='object').columns:
    if 'unknown' in df[col].values:
        count = (df[col] == 'unknown').sum()
        pct = count / len(df) * 100
        print(f"  {col}: {count:,} ({pct:.1f}%)")

In [ ]:
# Le taux de souscription des 'unknown' est-il différent ?
print("Taux de souscription pour les valeurs 'unknown' :")
for col in ['job', 'marital', 'education', 'default', 'housing', 'loan']:
    if 'unknown' in df[col].values:
        unknown_rate = (df[df[col] == 'unknown']['y'] == 'yes').mean() * 100
        known_rate = (df[df[col] != 'unknown']['y'] == 'yes').mean() * 100
        print(f"  {col}: unknown={unknown_rate:.1f}% vs known={known_rate:.1f}%")

print("\n→ Les 'unknown' peuvent être gardés comme catégorie à part (pas de suppression nécessaire)")

## 9. Résumé des insights pour la modélisation

### Features à exclure
- **`duration`** : Data leakage (connue après l'appel)

### Features les plus prédictives (hypothèses à valider)
1. **poutcome** : Succès précédent → fort prédicteur
2. **Indicateurs économiques** : euribor3m, nr.employed
3. **month** : Saisonnalité forte (mars, sept, oct, déc)
4. **age** : Jeunes et seniors plus réceptifs
5. **job** : Étudiants et retraités
6. **campaign** : Effet négatif après 3-4 contacts

### Recommandations métier
1. Cibler en priorité les clients ayant déjà souscrit
2. Concentrer les campagnes en automne (sept-oct-déc)
3. Limiter à 3-4 tentatives par client
4. Cibler les étudiants et retraités
5. Intensifier en période d'incertitude économique

### Déséquilibre des classes
- Ratio ~1:8 (11% vs 89%)
- Métriques : F1-score, Precision, Recall, AUC-ROC
- Techniques : SMOTE, class weights, undersampling

In [ ]:
# Nettoyage : supprimer les colonnes temporaires
df = df.drop(columns=['age_group', 'campaign_group', 'y_numeric'], errors='ignore')

print("✅ Exploration terminée.")
print(f"\nDataset final : {df.shape[0]:,} lignes, {df.shape[1]} colonnes")
print(f"Features à utiliser : {df.shape[1] - 1} (sans 'y')")
print(f"Feature à exclure pour le modèle réaliste : 'duration'")